In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('Data/Nat_Gas.csv')
df['Dates'] = pd.to_datetime(df['Dates'])
df.head()

C:\Users\Jian Jin Chen\AppData\Local\Temp\ipykernel_11336\2415161062.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])


,Dates,Prices
0,2020-10-31,10.1
1,2020-11-30,10.3
2,2020-12-31,11.0
3,2021-01-31,10.9
4,2021-02-28,10.9


In [4]:
def price_gas_storage_contract(
    injection_dates,
    injection_volumes,
    withdrawal_dates,
    withdrawal_volumes,
    price_data,
    max_storage_volume,
    storage_cost_per_month,
    injection_cost_per_mmbtu,
    withdrawal_cost_per_mmbtu,
    injection_rate_mmbtu_per_day=None,
    withdrawal_rate_mmbtu_per_day=None
):
    """
    Price a natural gas storage contract.
    
    Parameters:
    -----------
    injection_dates : list of datetime or str
        Dates when gas is injected into storage
    injection_volumes : list of float
        Volumes (MMBtu) to inject on each injection date
    withdrawal_dates : list of datetime or str
        Dates when gas is withdrawn from storage
    withdrawal_volumes : list of float
        Volumes (MMBtu) to withdraw on each withdrawal date
    price_data : DataFrame
        DataFrame with columns ['Dates', 'Prices'] containing historical/forward prices
    max_storage_volume : float
        Maximum storage capacity in MMBtu
    storage_cost_per_month : float
        Monthly fixed storage cost in dollars
    injection_cost_per_mmbtu : float
        Cost per MMBtu for injection
    withdrawal_cost_per_mmbtu : float
        Cost per MMBtu for withdrawal
    injection_rate_mmbtu_per_day : float, optional
        Maximum injection rate (MMBtu per day)
    withdrawal_rate_mmbtu_per_day : float, optional
        Maximum withdrawal rate (MMBtu per day)
    
    Returns:
    --------
    dict : Dictionary containing:
        - 'contract_value': Net value of the contract
        - 'total_revenue': Total revenue from selling gas
        - 'total_purchase_cost': Total cost of purchasing gas
        - 'total_injection_cost': Total injection costs
        - 'total_withdrawal_cost': Total withdrawal costs
        - 'total_storage_cost': Total storage costs
        - 'volume_check': Boolean indicating if storage constraints are satisfied
        - 'breakdown': DataFrame with detailed cash flow breakdown
    """
    
    # Convert dates to datetime if needed
    injection_dates = pd.to_datetime(injection_dates)
    withdrawal_dates = pd.to_datetime(withdrawal_dates)
    
    # Validate inputs
    if len(injection_dates) != len(injection_volumes):
        raise ValueError("Number of injection dates must match number of injection volumes")
    if len(withdrawal_dates) != len(withdrawal_volumes):
        raise ValueError("Number of withdrawal dates must match number of withdrawal volumes")
    
    # Check if total injection equals total withdrawal
    total_injection = sum(injection_volumes)
    total_withdrawal = sum(withdrawal_volumes)
    if not np.isclose(total_injection, total_withdrawal):
        print(f"Warning: Total injection ({total_injection}) != Total withdrawal ({total_withdrawal})")
    
    # Check storage capacity
    storage_profile = []
    current_volume = 0
    all_dates = sorted(list(injection_dates) + list(withdrawal_dates))
    
    for date in all_dates:
        if date in injection_dates.values:
            idx = list(injection_dates).index(date)
            current_volume += injection_volumes[idx]
        if date in withdrawal_dates.values:
            idx = list(withdrawal_dates).index(date)
            current_volume -= withdrawal_volumes[idx]
        storage_profile.append((date, current_volume))
    
    max_volume_used = max([vol for _, vol in storage_profile])
    volume_check = max_volume_used <= max_storage_volume
    
    if not volume_check:
        print(f"Warning: Maximum volume used ({max_volume_used}) exceeds storage capacity ({max_storage_volume})")
    
    # Create price lookup dictionary
    price_dict = dict(zip(pd.to_datetime(price_data['Dates']), price_data['Prices']))
    
    # Calculate purchase costs (injection dates)
    total_purchase_cost = 0
    injection_cash_flows = []
    
    for date, volume in zip(injection_dates, injection_volumes):
        if date in price_dict:
            price = price_dict[date]
            cost = price * volume
            total_purchase_cost += cost
            injection_cash_flows.append({
                'Date': date,
                'Type': 'Purchase',
                'Volume': volume,
                'Price': price,
                'Cash_Flow': -cost
            })
        else:
            print(f"Warning: No price data for injection date {date}")
    
    # Calculate injection costs
    total_injection_cost = sum(injection_volumes) * injection_cost_per_mmbtu
    injection_cost_flows = [{
        'Date': date,
        'Type': 'Injection_Cost',
        'Volume': volume,
        'Price': injection_cost_per_mmbtu,
        'Cash_Flow': -volume * injection_cost_per_mmbtu
    } for date, volume in zip(injection_dates, injection_volumes)]
    
    # Calculate revenue (withdrawal dates)
    total_revenue = 0
    withdrawal_cash_flows = []
    
    for date, volume in zip(withdrawal_dates, withdrawal_volumes):
        if date in price_dict:
            price = price_dict[date]
            revenue = price * volume
            total_revenue += revenue
            withdrawal_cash_flows.append({
                'Date': date,
                'Type': 'Sale',
                'Volume': volume,
                'Price': price,
                'Cash_Flow': revenue
            })
        else:
            print(f"Warning: No price data for withdrawal date {date}")
    
    # Calculate withdrawal costs
    total_withdrawal_cost = sum(withdrawal_volumes) * withdrawal_cost_per_mmbtu
    withdrawal_cost_flows = [{
        'Date': date,
        'Type': 'Withdrawal_Cost',
        'Volume': volume,
        'Price': withdrawal_cost_per_mmbtu,
        'Cash_Flow': -volume * withdrawal_cost_per_mmbtu
    } for date, volume in zip(withdrawal_dates, withdrawal_volumes)]
    
    # Calculate storage costs
    # Storage cost is charged for the duration gas is in storage
    if len(all_dates) >= 2:
        start_date = min(all_dates)
        end_date = max(all_dates)
        num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
        # Add 1 to include both start and end months
        num_months = max(1, num_months + 1)
    else:
        num_months = 1
    
    total_storage_cost = storage_cost_per_month * num_months
    storage_cost_flow = {
        'Date': min(all_dates) if all_dates else pd.Timestamp.now(),
        'Type': 'Storage_Cost',
        'Volume': num_months,
        'Price': storage_cost_per_month,
        'Cash_Flow': -total_storage_cost
    }
    
    # Calculate net contract value
    contract_value = (
        total_revenue 
        - total_purchase_cost 
        - total_injection_cost 
        - total_withdrawal_cost 
        - total_storage_cost
    )
    
    # Create detailed breakdown
    all_flows = (
        injection_cash_flows + 
        injection_cost_flows + 
        withdrawal_cash_flows + 
        withdrawal_cost_flows + 
        [storage_cost_flow]
    )
    breakdown_df = pd.DataFrame(all_flows)
    breakdown_df = breakdown_df.sort_values('Date').reset_index(drop=True)
    
    # Return results
    results = {
        'contract_value': contract_value,
        'total_revenue': total_revenue,
        'total_purchase_cost': total_purchase_cost,
        'total_injection_cost': total_injection_cost,
        'total_withdrawal_cost': total_withdrawal_cost,
        'total_storage_cost': total_storage_cost,
        'storage_months': num_months,
        'max_volume_used': max_volume_used,
        'volume_check': volume_check,
        'breakdown': breakdown_df
    }
    
    return results


def print_contract_summary(results):
    """
    Print a formatted summary of the contract valuation.
    """
    print("=" * 60)
    print("GAS STORAGE CONTRACT VALUATION SUMMARY")
    print("=" * 60)
    print(f"\nRevenue from gas sales:        ${results['total_revenue']:,.2f}")
    print(f"Cost of gas purchases:         ${results['total_purchase_cost']:,.2f}")
    print(f"Injection costs:               ${results['total_injection_cost']:,.2f}")
    print(f"Withdrawal costs:              ${results['total_withdrawal_cost']:,.2f}")
    print(f"Storage costs ({results['storage_months']} months):       ${results['total_storage_cost']:,.2f}")
    print("-" * 60)
    print(f"NET CONTRACT VALUE:            ${results['contract_value']:,.2f}")
    print("=" * 60)
    print(f"\nStorage capacity check: {'PASSED' if results['volume_check'] else 'FAILED'}")
    print(f"Maximum volume used: {results['max_volume_used']:,.0f} MMBtu")
    print("\n")

In [6]:
# Test Case 1: Simple single injection/withdrawal scenario
# (Similar to the example in your description)

# Assuming you have price data loaded as 'df'
# Create sample injection and withdrawal scenarios

print("TEST CASE 1: Single Injection/Withdrawal")
print("-" * 60)

# Inject 1 million MMBtu in summer at $2/MMBtu
# Withdraw 1 million MMBtu in winter at $3/MMBtu
# Storage cost: $100K per month for 4 months
# Injection/withdrawal cost: $10K per million MMBtu

injection_dates_1 = ['2023-06-01']
injection_volumes_1 = [1_000_000]  # 1 million MMBtu

withdrawal_dates_1 = ['2023-10-01']
withdrawal_volumes_1 = [1_000_000]  # 1 million MMBtu

# Create sample price data for this test
sample_prices = pd.DataFrame({
    'Dates': pd.to_datetime(['2023-06-01', '2023-10-01']),
    'Prices': [2.0, 3.0]  # $2 in summer, $3 in winter
})

results_1 = price_gas_storage_contract(
    injection_dates=injection_dates_1,
    injection_volumes=injection_volumes_1,
    withdrawal_dates=withdrawal_dates_1,
    withdrawal_volumes=withdrawal_volumes_1,
    price_data=sample_prices,
    max_storage_volume=2_000_000,  # 2 million MMBtu capacity
    storage_cost_per_month=100_000,  # $100K per month
    injection_cost_per_mmbtu=0.01,  # $10K per million = $0.01 per MMBtu
    withdrawal_cost_per_mmbtu=0.01  # $10K per million = $0.01 per MMBtu
)

print_contract_summary(results_1)
print("Detailed Cash Flow Breakdown:")
print(results_1['breakdown'])
print("\n" * 2)


# Test Case 2: Multiple injections and withdrawals

print("TEST CASE 2: Multiple Injections/Withdrawals")
print("-" * 60)

# Multiple injection dates in summer (lower prices)
injection_dates_2 = ['2023-05-01', '2023-06-01', '2023-07-01']
injection_volumes_2 = [300_000, 500_000, 200_000]  # Total 1M MMBtu

# Multiple withdrawal dates in winter (higher prices)
withdrawal_dates_2 = ['2023-11-01', '2023-12-01', '2024-01-01']
withdrawal_volumes_2 = [400_000, 400_000, 200_000]  # Total 1M MMBtu

sample_prices_2 = pd.DataFrame({
    'Dates': pd.to_datetime(['2023-05-01', '2023-06-01', '2023-07-01', 
                             '2023-11-01', '2023-12-01', '2024-01-01']),
    'Prices': [1.8, 2.0, 2.1, 3.2, 3.5, 3.8]  # Lower in summer, higher in winter
})

results_2 = price_gas_storage_contract(
    injection_dates=injection_dates_2,
    injection_volumes=injection_volumes_2,
    withdrawal_dates=withdrawal_dates_2,
    withdrawal_volumes=withdrawal_volumes_2,
    price_data=sample_prices_2,
    max_storage_volume=1_500_000,  # 1.5 million MMBtu capacity
    storage_cost_per_month=80_000,  # $80K per month
    injection_cost_per_mmbtu=0.015,  # $15K per million MMBtu
    withdrawal_cost_per_mmbtu=0.012  # $12K per million MMBtu
)

print_contract_summary(results_2)
print("Detailed Cash Flow Breakdown:")
print(results_2['breakdown'])
print("\n" * 2)


# Test Case 3: Using your actual price data from the CSV

print("TEST CASE 3: Using Actual Market Data")
print("-" * 60)

# First, check how many rows we have
print(f"Total rows in dataset: {len(df)}")
print(f"Date range: {df['Dates'].min()} to {df['Dates'].max()}")

# Select actual dates from your data
# Inject during low price period, withdraw during high price period
# Using valid indices (0-47)
injection_dates_3 = [df.iloc[10]['Dates'], df.iloc[20]['Dates']]
injection_volumes_3 = [600_000, 400_000]

withdrawal_dates_3 = [df.iloc[30]['Dates'], df.iloc[40]['Dates']]  # Changed from 50 and 60
withdrawal_volumes_3 = [700_000, 300_000]

print(f"\nInjection dates: {injection_dates_3}")
print(f"Injection prices: ${df.iloc[10]['Prices']:.2f}, ${df.iloc[20]['Prices']:.2f}")
print(f"\nWithdrawal dates: {withdrawal_dates_3}")
print(f"Withdrawal prices: ${df.iloc[30]['Prices']:.2f}, ${df.iloc[40]['Prices']:.2f}\n")

results_3 = price_gas_storage_contract(
    injection_dates=injection_dates_3,
    injection_volumes=injection_volumes_3,
    withdrawal_dates=withdrawal_dates_3,
    withdrawal_volumes=withdrawal_volumes_3,
    price_data=df,
    max_storage_volume=1_000_000,
    storage_cost_per_month=50_000,
    injection_cost_per_mmbtu=0.02,
    withdrawal_cost_per_mmbtu=0.02
)

print_contract_summary(results_3)
print("\nDetailed Cash Flow Breakdown:")
print(results_3['breakdown'])

TEST CASE 1: Single Injection/Withdrawal
------------------------------------------------------------
GAS STORAGE CONTRACT VALUATION SUMMARY

Revenue from gas sales:        $3,000,000.00
Cost of gas purchases:         $2,000,000.00
Injection costs:               $10,000.00
Withdrawal costs:              $10,000.00
Storage costs (5 months):       $500,000.00
------------------------------------------------------------
NET CONTRACT VALUE:            $480,000.00

Storage capacity check: PASSED
Maximum volume used: 1,000,000 MMBtu


Detailed Cash Flow Breakdown:
        Date             Type   Volume      Price  Cash_Flow
0 2023-06-01         Purchase  1000000       2.00 -2000000.0
1 2023-06-01   Injection_Cost  1000000       0.01   -10000.0
2 2023-06-01     Storage_Cost        5  100000.00  -500000.0
3 2023-10-01             Sale  1000000       3.00  3000000.0
4 2023-10-01  Withdrawal_Cost  1000000       0.01   -10000.0



TEST CASE 2: Multiple Injections/Withdrawals
---------------------